Importando babriogecas

In [ ]:
import numpy as np
import pandas as pd
import folium
import geopandas as gpd
from sqlalchemy import create_engine
from folium import plugins

tiles_list = ['Stamen Terrain', 'Stamen Toner', 'Stamen Water Color', 'cartodbpositron', 'cartodbdark_matter']

Configuração do banco

In [ ]:
host = "localhost:5432"
database = "db_sig"
user = 'postgres'
password = 'pedro123'

connection_string = f"postgresql+psycopg2://{user}:{password}@{host}/{database}"

engine = create_engine(connection_string)

Questão 1 - Distinção de objetos por meio de seus atributos qualitativos (nominal ou ordinal) por meio de cores

In [ ]:
sql = "select * from macrorregioes_hidrograficas"
df_macro_hidro = gpd.read_postgis(sql, engine, geom_col='area')

df_macro_hidro

In [ ]:


map_macro_hidro = folium.Map([-15.783333, -47.866667],
                 zoom_start=5,
                 )

gjson_macro_hidro = df_macro_hidro.__geo_interface__

cor_macrorregioes = {
    'AMAZÔNICA': 'purple',
    'TOCANTINS-ARAGUAIA': 'blue',
    'ATLÂNTICO NORDESTE OCIDENTAL': 'red',
    'PARNAÍBA': 'yellow',
    'ATLÂNTICO NORDESTE ORIENTAL': 'pink',
    'SÃO FRANCISCO': 'orange',
    'ATLÂNTICO LESTE': 'green',
    'ATLÂNTICO SUDESTE': 'cyan',
    'ATLÂNTICO SUL': 'magenta',
    'URUGUAI': 'brown',
    'PARANÁ': 'teal',
    'PARAGUAI': 'navy',
}

style = lambda feature: {
    'fillColor': cor_macrorregioes[feature['properties']['nm_macrorh']],
    'color': 'black',
    'fillOpacity': 0.5,
    'weight': 1.0,
}

area_macro_hidro = folium.GeoJson(gjson_macro_hidro, name='Macrorregiões hidrográficas', style_function = style)

map_macro_hidro.add_child(area_macro_hidro)

# Adicionando controlador de camadas
for tile in tiles_list:
    folium.TileLayer(tile, overlay = False, control = True).add_to(map_macro_hidro)
folium.LayerControl().add_to(map_macro_hidro)

map_macro_hidro.save('macrorregioes_hidrograficas.html')
map_macro_hidro


Questão 2 - Distinção de objetos por meio de seus atributos quantitativos (numéricos) por meio de escala de cores (pontos, linhas ou áreas) ou raios (pontos)

In [ ]:
sql = "select * from taxa_mortalidade"
df_taxa_mort = gpd.read_postgis(sql, engine)

df_taxa_mort

In [ ]:
map_taxa_mort = folium.Map([-15.783333, -47.866667],
                  zoom_start=5,
                  
                   )

gjson_taxa_mort = df_taxa_mort.__geo_interface__

folium.Choropleth(
    geo_data=gjson_taxa_mort,
    name="choropleth",
    data=df_taxa_mort,
    columns=["nome", "valor"],
    key_on="feature.properties.nome",
    fill_color="BuPu",
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name=" Taxa de mortalidade atribuída a doenças cardiovasculares, câncer, diabetes ou doenças respiratórias crônicas por região geográfica-2014(%)",  
).add_to(map_taxa_mort)

# Adicionando controlador de camadas
for tile in tiles_list:
    folium.TileLayer(tile, overlay = False, control = True).add_to(map_taxa_mort)
folium.LayerControl().add_to(map_taxa_mort)

map_taxa_mort.save('taxa_mortalidade.html')
map_taxa_mort

Questão 3 - Agrupamento de recursos pontuais ou heatmaps;

In [ ]:
sql = "select * from bovinos"
df_bovinos = gpd.read_postgis(sql, engine, geom_col='geom')

In [ ]:
map_bovinos = folium.Map([-15.783333, -47.866667],
                  zoom_start=5
                   )

df_bovinos['latitude'] = df_bovinos['geom'].y.astype(float)
df_bovinos['longitude'] = df_bovinos['geom'].x.astype(float)

# Usa a quantidade de bovinos como peso
heat_data1 = [[row['latitude'], row['longitude'], row['bovino2008']] for index, row in df_bovinos.iterrows()]

plugins.HeatMap(heat_data1,
        name = 'Quantidade de bovinos em 2008',
        blur=5,
        radius=6).add_to(map_bovinos)

# Adicionando controlador de camadas
for tile in tiles_list:
    folium.TileLayer(tile, overlay = False, control = True).add_to(map_bovinos)
folium.LayerControl().add_to(map_bovinos)

map_bovinos.save('bovinos.html')
map_bovinos

Questão 4 - Exibir pelo menos uma camada que seja resultado de junções espaciais.

In [ ]:
sql = """
select et.id as id_empresa,
	    et.geom as geom_empresa,
		e.geom as geom_maranhao
from empresas_transporte_carga_ferroviaria et, estados e 
where st_within(et.geom, e.geom) 
and e.nm_uf = 'Maranhão' 
"""
df_empresas_transporte_ma = gpd.read_postgis(sql, engine, geom_col='geom_empresa')

df_empresas_transporte_ma